# Input and output formats

This section provides an overview of the input and output formats
supported by DFTK, usually via integration with a third-party library.

## Reading / writing files supported by AtomsIO
[AtomsIO](https://github.com/mfherbst/AtomsIO.jl) is a Julia package which supports
reading / writing atomistic structures from / to a large range of file formats.
Supported formats include Crystallographic Information Framework (CIF),
XYZ and extxyz files, ASE / Gromacs / LAMMPS / Amber trajectory files
or input files of various other codes (e.g. Quantum Espresso, VASP, ABINIT, CASTEP, …).
The full list of formats is is available in the
[AtomsIO documentation](https://mfherbst.github.io/AtomsIO.jl/stable).

The AtomsIO functionality is split into two packages. The main package, `AtomsIO` itself,
only depends on packages, which are registered in the Julia General package registry.
In contrast `AtomsIOPython` extends `AtomsIO` by parsers depending on python packages,
which are automatically managed via `PythonCall`. While it thus provides the full set of
supported IO formats, this also adds additional practical complications, so some users
may choose not to use `AtomsIOPython`.

As an example we start the calculation of a simple antiferromagnetic iron crystal
using a Quantum-Espresso input file, [Fe_afm.pwi](Fe_afm.pwi).
For more details about calculations on magnetic systems
using collinear spin, see Collinear spin and magnetic systems.

First we parse the Quantum Espresso input file using AtomsIO,
which reads the lattice, atomic positions and initial magnetisation
from the input file and returns it as an
[AtomsBase](https://github.com/JuliaMolSim/AtomsBase.jl) `AbstractSystem`,
the JuliaMolSim community standard for representing atomic systems.

In [1]:
using AtomsIO        # Use Julia-only IO parsers
using AtomsIOPython  # Use python-based IO parsers (e.g. ASE)
system = load_system("Fe_afm.pwi")

FlexibleSystem(Fe₂, periodic = TTT):
    bounding_box      : [ 2.86814        0        0;
                                0  2.86814        0;
                                0        0  2.86814]u"Å"

    Atom(Fe, [       0,        0,        0]u"Å")
    Atom(Fe, [ 1.43407,  1.43407,  1.43407]u"Å")

      .------.  
     /|      |  
    * |      |  
    | | Fe   |  
    | .------.  
    |/      /   
    Fe-----*    


Next we attach pseudopotential information, since currently the parser is not
yet capable to read this information from the file.

In [2]:
using DFTK
system = attach_psp(system, Fe="hgh/pbe/fe-q16.hgh")

FlexibleSystem(Fe₂, periodic = TTT):
    bounding_box      : [ 2.86814        0        0;
                                0  2.86814        0;
                                0        0  2.86814]u"Å"

    Atom(Fe, [       0,        0,        0]u"Å")
    Atom(Fe, [ 1.43407,  1.43407,  1.43407]u"Å")

      .------.  
     /|      |  
    * |      |  
    | | Fe   |  
    | .------.  
    |/      /   
    Fe-----*    


Finally we make use of DFTK's AtomsBase integration to run the calculation.

In [3]:
model = model_LDA(system; temperature=0.01)
basis = PlaneWaveBasis(model; Ecut=10, kgrid=(2, 2, 2))
ρ0 = guess_density(basis, system)
scfres = self_consistent_field(basis, ρ=ρ0);

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   Diag   Δtime
---   ---------------   ---------   ---------   ------   ----   ------
  1   -224.0551535606                   -0.05   -0.653    5.1    342ms
  2   -224.1931336174       -0.86       -0.46   -0.551    1.8    207ms
  3   -224.2168835758       -1.62       -1.71   -0.489    3.0    263ms
  4   -224.2169598570       -4.12       -2.45   -0.467    1.0    155ms
  5   -224.2169701754       -4.99       -2.63   -0.448    1.8    172ms
  6   -224.2169788478       -5.06       -2.73   -0.411    1.5    168ms
  7   -224.2169959968       -4.77       -2.74   -0.291    1.5    179ms
  8   -224.2170085720       -4.90       -2.78   -0.146    2.0    202ms
  9   -224.2170127935       -5.37       -2.94   -0.037    1.8    198ms
 10   -224.2170133590       -6.25       -3.19   -0.014    1.0    155ms
 11   -224.2170136172       -6.59       -3.55   -0.005    1.4    169ms
 12   -224.2170136726       -7.26       -4.16    0.001    1.5    171ms
 13   -

!!! warning "DFTK data formats are not yet fully matured"
    The data format in which DFTK saves data as well as the general interface
    of the `load_scfres` and `save_scfres` pair of functions
    are not yet fully matured. If you use the functions or the produced files
    expect that you need to adapt your routines in the future even with patch
    version bumps.

## Writing VTK files for visualization
For visualizing the density or the Kohn-Sham orbitals DFTK supports storing
the result of an SCF calculations in the form of VTK files.
These can afterwards be visualized using tools such
as [paraview](https://www.paraview.org/).
Using this feature requires
the [WriteVTK.jl](https://github.com/jipolanco/WriteVTK.jl/) Julia package.

In [4]:
using WriteVTK
save_scfres("iron_afm.vts", scfres; save_ψ=true);

This will save the iron calculation above into the file `iron_afm.vts`,
using `save_ψ=true` to also include the KS orbitals.

## Parsable data-export using json
Many structures in DFTK support the (unexported) `todict` function,
which returns a simplified dictionary representation of the data.

In [5]:
DFTK.todict(scfres.energies)

Dict{String, Float64} with 9 entries:
  "AtomicNonlocal" => -4.32427
  "PspCorrection"  => 7.03001
  "Ewald"          => -171.89
  "total"          => -224.217
  "Entropy"        => -0.0328468
  "Kinetic"        => 74.1104
  "AtomicLocal"    => -139.801
  "Hartree"        => 31.6017
  "Xc"             => -20.9115

This in turn can be easily written to disk using a JSON library.
Currently we integrate most closely with `JSON3`,
which is thus recommended.

In [6]:
using JSON3
open("iron_afm_energies.json", "w") do io
    JSON3.pretty(io, DFTK.todict(scfres.energies))
end
println(read("iron_afm_energies.json", String))

{
    "AtomicNonlocal": -4.324270060193098,
    "PspCorrection": 7.03000636467455,
    "Ewald": -171.88999401525265,
    "total": -224.217013677285,
    "Entropy": -0.032846804697391145,
    "Kinetic": 74.11040770257233,
    "AtomicLocal": -139.80051059333138,
    "Hartree": 31.601692893380438,
    "Xc": -20.911499164437796
}


Once JSON3 is loaded, additionally a convenience function for saving
a summary of `scfres` objects using `save_scfres` is available:

In [7]:
using JSON3
save_scfres("iron_afm.json", scfres)

Similarly a summary of the band data (occupations, eigenvalues, εF, etc.)
for post-processing can be dumped using `save_bands`:

In [8]:
save_bands("iron_afm_scfres.json", scfres)

Notably this function works both for the results obtained
by `self_consistent_field` as well as `compute_bands`:

In [9]:
bands = compute_bands(scfres, kline_density=10)
save_bands("iron_afm_bands.json", bands)

┌ Warning: The provided cell is a supercell: the returned k-path is the standard k-path of the associated primitive cell in the basis of the supercell reciprocal lattice.
│   cell =
│    Spglib.SpglibCell{Float64, Float64, Int64, Float64}
│     lattice:
│       5.41999997388764  0.0  0.0
│       0.0  5.41999997388764  0.0
│       0.0  0.0  5.41999997388764
│     2 atomic positions:
│       0.0  0.0  0.0
│       0.5  0.5  0.5
│     2 atoms:
│       1  1
│     2 magmoms:
│       0.0  0.0
│    
└ @ BrillouinSpglibExt ~/.julia/packages/Brillouin/5Kz8P/ext/BrillouinSpglibExt.jl:28


## Writing and reading JLD2 files
The full state of a DFTK self-consistent field calculation can be
stored on disk in form of an [JLD2.jl](https://github.com/JuliaIO/JLD2.jl) file.
This file can be read from other Julia scripts
as well as other external codes supporting the HDF5 file format
(since the JLD2 format is based on HDF5). This includes notably `h5py`
to read DFTK output from python.

In [10]:
using JLD2
save_scfres("iron_afm.jld2", scfres)

Saving such JLD2 files supports some options, such as `save_ψ=false`, which avoids saving
the Bloch waves (much faster and smaller files). Notice that JLD2 files can also be used
with `save_bands`.

Since such JLD2 can also be read by DFTK to start or continue a calculation,
these can also be used for checkpointing or for transferring results
to a different computer.
See Saving SCF results on disk and SCF checkpoints for details.

(Cleanup files generated by this notebook.)

In [11]:
rm.(["iron_afm.vts", "iron_afm.jld2",
     "iron_afm.json", "iron_afm_energies.json", "iron_afm_scfres.json",
     "iron_afm_bands.json"])

6-element Vector{Nothing}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing